## Recommendation with Distance


In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.multiclass import OneVsRestClassifier
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import pairwise_distances
import random
from numpy import inf
from numpy import linalg as LA


In [2]:
dataset = pd.read_csv("features_msd_lda_sp.csv")
dataset.drop(["Unnamed: 0"], 1, inplace=True)
dataset.columns

Index(['genre', 'track_id', 'artist_name', 'title', 'loudness_x', 'tempo_x',
       'time_signature', 'key_x', 'mode_x', 'duration', 'avg_timbre1',
       'avg_timbre2', 'avg_timbre3', 'avg_timbre4', 'avg_timbre5',
       'avg_timbre6', 'avg_timbre7', 'avg_timbre8', 'avg_timbre9',
       'avg_timbre10', 'avg_timbre11', 'avg_timbre12', 'var_timbre1',
       'var_timbre2', 'var_timbre3', 'var_timbre4', 'var_timbre5',
       'var_timbre6', 'var_timbre7', 'var_timbre8', 'var_timbre9',
       'var_timbre10', 'var_timbre11', 'var_timbre12', '0', '1', '2', '3', '4',
       '5', '6', '7', '8', '9', 'danceability', 'energy', 'key_y',
       'loudness_y', 'mode_y', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo_y', 'id'],
      dtype='object')

## Recommendation with distance

Let's change the genre to the genre id

In [3]:
genres = dataset.genre.unique()
dataset.genre.replace(genres, np.arange(10), inplace=True)
dataset



,genre,track_id,artist_name,title,loudness_x,tempo_x,time_signature,key_x,mode_x,duration,...,key_y,loudness_y,mode_y,speechiness,acousticness,instrumentalness,liveness,valence,tempo_y,id
0,0,TRNJTPB128F427AE9F,Blue Oyster Cult,Screams,-10.659,148.462,1,4,0,189.80526,...,4,-9.009,0,0.0806,0.32600,0.014300,0.2240,0.4930,149.356,2KsnSzoJlmfT44jFOqRP1E
1,0,TRLFJHA128F427AEEA,Blue Oyster Cult,Dance The Night Away,-13.494,112.909,1,10,0,158.19710,...,10,-11.838,0,0.0359,0.90300,0.000000,0.1810,0.1930,80.510,4qV4ErzHMz4XYGWxMkOFBw
2,0,TRCQZAG128F427DB97,Blue Oyster Cult,Debbie Denise,-12.786,117.429,4,7,1,250.22649,...,7,-7.034,1,0.0303,0.36100,0.002490,0.1170,0.4670,117.720,2o2jnahWDsvyo8v0vV3dif
3,0,TRSIZRN128F427DB95,Blue Oyster Cult,Morning Final,-11.952,100.901,1,0,1,254.17098,...,9,-6.756,0,0.0659,0.05170,0.033700,0.1520,0.5050,100.563,4YsdlcE3GqCHpZD4jNDJgD
4,0,TRDYTEO128F427DB90,Blue Oyster Cult,The Revenge Of Vera Gemini,-11.839,132.361,4,2,1,230.32118,...,9,-7.110,1,0.0659,0.00182,0.000002,0.0814,0.6570,132.659,4aumrXau5uVKj8xXcQELP3
5,0,TRKSICM128F427DB8B,Blue Oyster Cult,True Confessions,-13.760,121.001,4,10,1,177.55383,...,10,-7.493,1,0.0433,0.10200,0.000009,0.0539,0.8140,121.065,6sH5tkQoXL3YKV0hpXZoWj
6,0,TRJPXIV128F426697A,Blue Oyster Cult,Redeemed,-10.799,185.836,4,2,1,231.39220,...,11,-9.794,0,0.0374,0.08460,0.001370,0.3210,0.5170,186.627,1P37l3UBUo49cqATq6Qccz
7,0,TRXWSIN128F4265A40,Blue Oyster Cult,Workshop Of The Telescopes,-11.413,120.171,4,9,1,241.16200,...,2,-8.954,1,0.1140,0.25700,0.011700,0.1110,0.3540,120.413,55f36XW2w1moYF70wsaHLB
8,0,TRNVQPE128F426BBD2,Blue Oyster Cult,Godzilla,-12.083,88.548,4,6,0,469.89016,...,4,-7.719,1,0.0554,0.16400,0.000008,0.6050,0.6960,184.024,6N0AnkdDFZUetw8KAGHV7e
9,0,TRUUZXH128F426C1AD,Blue Oyster Cult,E.T.I. (Extra Terrestrial Intelligence),-7.264,97.298,4,9,1,226.03710,...,9,-5.875,1,0.0486,0.24500,0.002490,0.4350,0.6860,97.335,5KBdHzTROSlD3dACh91sZx


In [4]:
node_id = dataset['id']
feature_set = dataset.drop(['track_id','artist_name','title','id'], 1)
feature_set.columns


Index(['genre', 'loudness_x', 'tempo_x', 'time_signature', 'key_x', 'mode_x',
       'duration', 'avg_timbre1', 'avg_timbre2', 'avg_timbre3', 'avg_timbre4',
       'avg_timbre5', 'avg_timbre6', 'avg_timbre7', 'avg_timbre8',
       'avg_timbre9', 'avg_timbre10', 'avg_timbre11', 'avg_timbre12',
       'var_timbre1', 'var_timbre2', 'var_timbre3', 'var_timbre4',
       'var_timbre5', 'var_timbre6', 'var_timbre7', 'var_timbre8',
       'var_timbre9', 'var_timbre10', 'var_timbre11', 'var_timbre12', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', 'danceability', 'energy',
       'key_y', 'loudness_y', 'mode_y', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo_y'],
      dtype='object')

Create distance matrix with all songs

In [5]:
for column in feature_set.columns:
    max_of_column = feature_set[column].max()
    feature_set[column] = feature_set[column].apply(lambda x: x / max_of_column)

In [26]:
distance_matrix = pairwise_distances(feature_set.loc[:],feature_set.loc[:],metric = 'l2')
real_distance_matrix = distance_matrix.copy()
distance_matrix = 1/distance_matrix 
distance_matrix[distance_matrix == inf ] = 0
total = distance_matrix.sum(axis=1)


Generate the cumulative distribution for each song

In [7]:
current = 0
def get_distribution(current):
    dists = distance_matrix[current]
    start = 0
    result = {}
    for s in range(len(dists)):
        if dists[s] != current:
            d_s = dists[s] / total[current]
            result[s] = (start, start + d_s)
            start += d_s

        if start >= 1:
            break
    
    return result


Create 10 seed songs by selecting randomly

In [8]:
seed_songs = feature_set.ix[np.random.choice(feature_set.index, 1)]
seed_songs

,genre,loudness_x,tempo_x,time_signature,key_x,mode_x,duration,avg_timbre1,avg_timbre2,avg_timbre3,...,energy,key_y,loudness_y,mode_y,speechiness,acousticness,instrumentalness,liveness,valence,tempo_y
7467,0.222222,-2.237347,0.519902,0.571429,0.0,1.0,0.093058,0.844543,0.067268,0.345078,...,0.774775,0.0,-1.336722,1.0,0.030737,0.178715,0.000645,0.395792,0.964753,0.561233


Find next song according to the current song and seed songs

In [9]:
def get_next_song(current,seed_songs):
    while current in seed_songs.index:
        distrib = get_distribution(current)
        r = random.random()
        for k, v in distrib.items():
            if v[0] <= r < v[1]:
                current = k
                break
    return current


In [10]:
get_recommended_songs = []
recommendation_count = 10
seed_song_index = seed_songs.index
seed_song = seed_song_index[0]
while recommendation_count > 0 : 
    next_song = get_next_song(seed_song,seed_songs)
    while next_song  in get_recommended_songs:
        next_song = get_next_song(seed_song,seed_songs)

    recommendation_count = recommendation_count -1
    get_recommended_songs.append(next_song)
get_recommended_songs
    


[7186, 6959, 2881, 534, 8174, 2581, 10267, 163, 3901, 3574]

Get id of recommendation songs according to the distance

In [11]:
recommended_songs = [(dataset.loc[i]['artist_name'], dataset.loc[i]['title'], dataset.loc[i]['id']) for i in get_recommended_songs]
recommended_songs

[('Lou Rhodes', 'Beloved One', '2cDHiYVbfwZ2dIcZGSQdfB'),
 ('Dan Bern', 'Tiger Woods', '1EI1rogiquJ3jg7PbHlvLI'),
 ('Trisha Yearwood', 'Pistol', '41WkYWObmdHoCeuBAj2gS3'),
 ('Jape', 'Floating', '00izCKePHFiaj89FKtvP27'),
 ('Patty Larkin', 'Louder', '3T8YrR6eOgha5UvDuxb7MD'),
 ('Lutricia Mcneal', 'Plano - Texas - USA', '692Qx7J7EI6xN3IKvcu6fR'),
 ('Overkill', "Long Time Dyin'", '6NIeo0O00EJ5ZotHk0bKwP'),
 ('Burning Spear', 'Follow Marcus Garvey', '5kSG1TK1Mpo0GuuAA8ap9y'),
 ('38 Special', 'Like No Other Night', '2I448HpxuWiSjmptMtJonH'),
 ('Big Star', "Whole Lotta Shakin' Goin' On", '5YcUJJIcXB4ibklrvhQKuw')]

Feature of Seed Song

In [12]:
seedSong = dataset.loc[seed_song]['artist_name'], dataset.loc[seed_song]['title'] ,dataset.loc[seed_song]['id'] 
seedSong

('Rhonda Vincent',
 'Heart Wrenching Lovesick Memories',
 '4iGwXavecgagKXOSkQgZZo')

## Evaluation of Recommendation

Look at the distance between seed song and other recommendation songs

In [24]:
seed_distance = []
for i in get_recommended_songs:
    print('Distance between artist name: ' +  dataset.loc[seed_song]['artist_name'] + ', title: ' + dataset.loc[seed_song]['title'] + ' and artist name: ' + dataset.loc[i]['artist_name'] + ', title: ' + dataset.loc[i]['title'])
    dist = LA.norm(feature_set.loc[seed_song].get_values() - feature_set.loc[i].get_values())
    seed_distance.append(dist)
    print(dist)
    print('\n')


Distance between artist name: Rhonda Vincent, title: Heart Wrenching Lovesick Memories and artist name: Lou Rhodes, title: Beloved One
2.7086531094854625


Distance between artist name: Rhonda Vincent, title: Heart Wrenching Lovesick Memories and artist name: Dan Bern, title: Tiger Woods
1.966101629584067


Distance between artist name: Rhonda Vincent, title: Heart Wrenching Lovesick Memories and artist name: Trisha Yearwood, title: Pistol
2.2999290710064137


Distance between artist name: Rhonda Vincent, title: Heart Wrenching Lovesick Memories and artist name: Jape, title: Floating
1.7578312661300362


Distance between artist name: Rhonda Vincent, title: Heart Wrenching Lovesick Memories and artist name: Patty Larkin, title: Louder
1.3525990687500606


Distance between artist name: Rhonda Vincent, title: Heart Wrenching Lovesick Memories and artist name: Lutricia Mcneal, title: Plano - Texas - USA
1.8985997330042097


Distance between artist name: Rhonda Vincent, title: Heart Wrenchi

Look at the distance between recommended songs 

In [29]:
distance = []
for i in range(len(get_recommended_songs)-1):
    first_song = get_recommended_songs[i]
    second_song = get_recommended_songs[i+1]    
    print('Distance between artist name: ' +  dataset.loc[first_song]['artist_name'] + ', title: ' + dataset.loc[first_song]['title'] + ' and artist name: ' + dataset.loc[second_song]['artist_name'] + ', title: ' + dataset.loc[second_song]['title'])
    dist = LA.norm(feature_set.loc[seed_song].get_values() - feature_set.loc[i].get_values())
    distance.append(dist)
    print(dist)
    print('\n')


Distance between artist name: Lou Rhodes, title: Beloved One and artist name: Dan Bern, title: Tiger Woods
2.683547456801936


Distance between artist name: Dan Bern, title: Tiger Woods and artist name: Trisha Yearwood, title: Pistol
3.978942608874901


Distance between artist name: Trisha Yearwood, title: Pistol and artist name: Jape, title: Floating
2.5725714295705773


Distance between artist name: Jape, title: Floating and artist name: Patty Larkin, title: Louder
2.675220556943089


Distance between artist name: Patty Larkin, title: Louder and artist name: Lutricia Mcneal, title: Plano - Texas - USA
2.377362959915185


Distance between artist name: Lutricia Mcneal, title: Plano - Texas - USA and artist name: Overkill, title: Long Time Dyin'
3.0706655605013298


Distance between artist name: Overkill, title: Long Time Dyin' and artist name: Burning Spear, title: Follow Marcus Garvey
2.603824312384177


Distance between artist name: Burning Spear, title: Follow Marcus Garvey and arti

Average distance between seed song and other recommended songs

In [25]:
np.mean(seed_distance)

1.9248961759047447

Average distance between recommended pairwise songs 

In [30]:
np.mean(distance)

2.79985410941185

Maximum distance in the dataset

In [28]:
np.amax(real_distance_matrix)  

17.91057970709778

Average distance in dataset

In [31]:
np.mean(real_distance_matrix)

2.93989764932603